### 하나의 mesh 파일에 대한 캡셔닝 진행하기

In [ ]:
import openai
import requests
import base64
import subprocess
import json
import time


# 타겟 오브젝트 로드
target_directory = "../outputs/mesh_segmentation_output/CustomDataset/000-0000"
config_directory = "../configs"

mesh_id = "01659373a49d4ff29e6b99abd3b734ce"  # 안테나?

matte_image_path = f"{target_directory}/{mesh_id}/matte_collage_2048x2048.png"
segmentation_image_path = f"{target_directory}/{mesh_id}/segmented_collage_2048x2048.png"

prompt_template_path = f"{config_directory}/captioning_template.txt"

with open('openai_api_key.txt', 'r') as file:
    api_key = file.read().strip()

# 텍스트 프롬프트 읽기
with open(prompt_template_path, 'r', encoding='utf-8') as f:
    prompt_template = f.read()

# 이미지를 base64로 인코딩
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

matte_image_b64 = encode_image_to_base64(matte_image_path)
segmentation_image_b64 = encode_image_to_base64(segmentation_image_path)

# LLM API 호출 함수
def call_openai_api(prompt, matte_image_b64, segmentation_image_b64):
    from openai import OpenAI

    client = OpenAI(api_key=api_key, timeout=30, max_retries=3)  # OPENAI_API_KEY 환경변수 필요

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": [
                {"type": "text", "text": "Here are two images of the object."},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{matte_image_b64}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{segmentation_image_b64}"}},
            ]}
        ]
    )
    return response.choices[0].message.content

def call_ollama_deepseek_api(prompt, matte_image_b64, segmentation_image_b64):
    url = "http://localhost:11434/api/generate"
    
    full_prompt = f"{prompt}\n\nMatte Image (base64): {matte_image_b64}\n\nSegmentation Image (base64): {segmentation_image_b64}\n\nPlease describe the 3D object's geometry based on these images."

    payload = {
        "model": "deepseek-r1:70b",  # 원하는 모델명으로 변경 가능
        "prompt": full_prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()['response']
    else:
        raise Exception(f"Ollama (DeepSeek) API 호출 실패: {response.status_code}, {response.text}")

def call_ollama_gemma_api(prompt, matte_image_b64, segmentation_image_b64):
    url = "http://localhost:11434/api/generate"

    payload = {
        "model": "gemma3:27b",
        "prompt": prompt,
        "images": [matte_image_b64, segmentation_image_b64],
        "stream": False
    }

    headers = {
        "Content-Type": "application/json"
    }

    max_retries = 3
    timeout_seconds = 30

    for attempt in range(1, max_retries + 1):
        try:
            print(f"[요청] Ollama Gemma API 호출 (시도 {attempt}/{max_retries})...")
            response = requests.post(url, headers=headers, json=payload, timeout=timeout_seconds)

            if response.status_code == 200:
                response_json = response.json()
                if "response" in response_json:
                    print("[성공] 요청 성공 ✅")
                    return response_json["response"]
                else:
                    raise Exception(f"[에러] 응답 형식 오류: {response_json}")
            else:
                raise Exception(f"[에러] HTTP 상태 코드 오류: {response.status_code}, {response.text}")

        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f"[경고] 요청 실패 (타임아웃 또는 연결 문제): {e}")
        except Exception as e:
            print(f"[경고] 요청 실패: {e}")

        if attempt < max_retries:
            wait_time = 2 ** attempt  # 지수 백오프 (2초 → 4초 → 8초)
            print(f"[대기] {wait_time}초 후 재시도합니다...\n")
            time.sleep(wait_time)
        else:
            print("[실패] 모든 재시도 실패 ❌")

    raise RuntimeError("최대 재시도 후에도 요청에 실패했습니다.")

# 사용 방법
use_openai = True
use_ollama_deepseek = False
use_ollama_gemma = False

# 호출 분기
if use_openai:
    result = call_openai_api(prompt_template, matte_image_path, segmentation_image_path)
elif use_ollama_deepseek:
    result = call_ollama_deepseek_api(prompt_template, matte_image_b64, segmentation_image_b64)
elif use_ollama_gemma:
    result = call_ollama_gemma_api(prompt_template, matte_image_b64, segmentation_image_b64)
else:
    raise ValueError("하나의 API만 True로 설정하세요.")

print("\n=== LLM 결과 ===")
print(result)


=== LLM 결과 ===
**Status:** Good

**Caption:**
A 3D model of a parabolic dish antenna featuring a concave dish with a central support structure extending from the back, connected to a base.

**Objects:**
Dish, Support Structure, Base

**Parts and Relationships:**
- The concave dish is attached to a central support structure.
- The support structure extends from the back of the dish.
- The support structure is connected to a base that provides stability.


### 특정 디렉토리 내에 있는 모든 메시 파일에 대해 캡셔닝 진행하기

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
batch_caption_meshes.py  (tqdm + 빈 디렉토리 처리 + 모델별 파일명)
"""
import os, sys, csv, base64, time, logging
from pathlib import Path
import requests
from tqdm import tqdm

# ───────────────────────────────────────────────────────────
# [1] 경로·설정
# ───────────────────────────────────────────────────────────
TARGET_DIR = Path("../outputs/mesh_segmentation_output/CustomDataset/000-001")
CONFIG_DIR = Path("../configs")
TEMPLATE   = CONFIG_DIR / "captioning_template.txt"

USE_OPENAI           = True
USE_OLLAMA_DEEPSEEK  = False
USE_OLLAMA_GEMMA     = False

CSV_PATH = TARGET_DIR / "caption_results.csv"
LOG_PATH = TARGET_DIR / "batch_caption_meshes.log"

with open('openai_api_key.txt', 'r') as file:
    api_key = file.read().strip()

# ───────────────────────────────────────────────────────────
# [2] 로깅
# ───────────────────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler(LOG_PATH, encoding="utf-8"),
              logging.StreamHandler(sys.stdout)],
)
log = logging.getLogger(__name__)

# ───────────────────────────────────────────────────────────
# [3] 헬퍼
# ───────────────────────────────────────────────────────────
def encode_b64(p: Path) -> str:
    with p.open("rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

PROMPT_TMPL = Path(TEMPLATE).read_text(encoding="utf-8")

def call_ollama_gemma(prompt: str, matte_b64: str, segm_b64: str,
                      *, retries=3, timeout=30) -> str:
    url = "http://localhost:11434/api/generate"
    payload = {"model": "gemma3:27b", "prompt": prompt,
               "images": [matte_b64, segm_b64], "stream": False}
    for a in range(1, retries + 1):
        try:
            r = requests.post(url, json=payload, timeout=timeout)
            r.raise_for_status()
            return r.json()["response"]
        except Exception as e:
            log.warning(f"[{a}/{retries}] Gemma 호출 실패: {e}")
            if a < retries:
                wait = 2 ** a
                log.info(f"{wait}초 뒤 재시도…")
                time.sleep(wait)
            else:
                raise
            
def call_openai_api(prompt, matte_image_b64, segmentation_image_b64):
    from openai import OpenAI

    client = OpenAI(api_key=api_key, timeout=30, max_retries=3)  # OPENAI_API_KEY 환경변수 필요

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": [
                {"type": "text", "text": "Here are two images of the object."},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{matte_image_b64}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{segmentation_image_b64}"}},
            ]}
        ]
    )
    return response.choices[0].message.content

# ───────────────────────────────────────────────────────────
# [4] 메인 루프
# ───────────────────────────────────────────────────────────
mesh_dirs = sorted(d for d in TARGET_DIR.iterdir() if d.is_dir())
results   = []

# ★변경: 어떤 모델을 쓰는지에 따라 파일 접미사 결정
if USE_OLLAMA_GEMMA:
    suffix = "gemma"
elif USE_OLLAMA_DEEPSEEK:
    suffix = "deepseek"
elif USE_OPENAI:
    suffix = "gpt"
else:
    suffix = "unknown"

for mesh_dir in tqdm(mesh_dirs, desc="Captioning meshes"):
    mesh_id      = mesh_dir.name
    matte_png    = mesh_dir / "matte_collage_2048x2048.png"
    segm_png     = mesh_dir / "segmented_collage_2048x2048.png"
    caption_path = mesh_dir / f"caption_{suffix}.txt"       # ★변경

    rec = {"mesh_id": mesh_id, "status": "success", "error_msg": ""}

    # ── 파일 존재 확인 ─────────────────────────────────────
    if not matte_png.exists() or not segm_png.exists():
        msg = "필요한 이미지 파일이 존재하지 않음"
        rec.update(status="error", error_msg=msg)
        log.warning(f"⚠️  {mesh_id}: {msg}")
        results.append(rec)
        continue

    try:
        matte_b64 = encode_b64(matte_png)
        segm_b64  = encode_b64(segm_png)

        if USE_OLLAMA_GEMMA:
            caption = call_ollama_gemma(PROMPT_TMPL, matte_b64, segm_b64)
        elif USE_OLLAMA_DEEPSEEK:
            caption = call_ollama_deepseek_api(PROMPT_TMPL, matte_b64, segm_b64)
        elif USE_OPENAI:
            caption = call_openai_api(PROMPT_TMPL, matte_b64, segm_b64)
        else:
            raise RuntimeError("LLM 플래그가 모두 False입니다.")

        caption_path.write_text(caption.strip() + "\n", encoding="utf-8")
        log.info(f"✅ {mesh_id} 완료 → {caption_path}")

    except Exception as e:
        rec.update(status="error", error_msg=str(e))
        log.error(f"❌ {mesh_id} 실패: {e}")

    results.append(rec)

# ───────────────────────────────────────────────────────────
# [5] CSV 저장
# ───────────────────────────────────────────────────────────
with CSV_PATH.open("w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["mesh_id", "status", "error_msg"])
    writer.writeheader()
    writer.writerows(results)

log.info(f"=== 전체 완료. CSV: {CSV_PATH} ===")

Captioning meshes:   0%|          | 0/386 [00:00<?, ?it/s]

2025-04-29 14:56:04,766 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-29 14:56:04,769 [INFO] ✅ 000234d39e684deb80b7b8d028681dc1 완료 → ../outputs/mesh_segmentation_output/CustomDataset/000-001/000234d39e684deb80b7b8d028681dc1/caption_gpt.txt


Captioning meshes:   0%|          | 1/386 [00:04<28:59,  4.52s/it]

2025-04-29 14:56:09,769 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-29 14:56:09,771 [INFO] ✅ 0073f94df6fe405eae5185fef6ba7b51 완료 → ../outputs/mesh_segmentation_output/CustomDataset/000-001/0073f94df6fe405eae5185fef6ba7b51/caption_gpt.txt


Captioning meshes:   1%|          | 2/386 [00:09<30:44,  4.80s/it]

...

2025-04-29 15:24:51,772 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-29 15:24:51,774 [INFO] ✅ y0KSBqo8lcEnAwqYkI29kgHCUn4 완료 → ../outputs/mesh_segmentation_output/CustomDataset/000-001/y0KSBqo8lcEnAwqYkI29kgHCUn4/caption_gpt.txt


Captioning meshes: 100%|█████████▉| 385/386 [28:51<00:04,  4.52s/it]

2025-04-29 15:24:55,373 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-29 15:24:55,375 [INFO] ✅ ydSFQC0HWE7eMaulss0Xak4pe1X 완료 → ../outputs/mesh_segmentation_output/CustomDataset/000-001/ydSFQC0HWE7eMaulss0Xak4pe1X/caption_gpt.txt


Captioning meshes: 100%|██████████| 386/386 [28:55<00:00,  4.50s/it]

2025-04-29 15:24:55,379 [INFO] === 전체 완료. CSV: ../outputs/mesh_segmentation_output/CustomDataset/000-001/caption_results.csv ===
